In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scripts.proj1_helpers import *
from scripts.preprocess import *

from scripts.split_data import *
from scripts.model_testing import *
from scripts.cross_validation import *

from scripts.implementations import least_squares_GD
from scripts.implementations import least_squares_SGD
from scripts.implementations import least_squares
from scripts.implementations import ridge_regression
from scripts.implementations import logistic_regression
from scripts.implementations import reg_logistic_regression

%load_ext autoreload
%autoreload 2

In [2]:
# load data
raw_y, raw_x, ind = load_csv_data('higgs-data/train.csv')

## * Data Processing
1. Based on PRI_JET_NUM (feature 22), which ranged in value of inclusive [0,4], we devide the training data into 4 sets. From each sets we obtain exactly one model (w0/w1/w2/w4). E.g. for PRI_JET_NUM=0 we will get w0.
2. We standardize the data using power terms.

### Creating Subsets

In [3]:
def create_subsets(x, y):
    sets_x = []
    sets_y = []
    for pri_jet_num_val in np.unique(x[:,22]):
        indices = (x[:,22] == pri_jet_num_val)
        x_tmp   = x[indices,:]
        y_tmp   = y[indices]
        
        sets_x.append(x_tmp)
        sets_y.append(y_tmp)
    return sets_x, sets_y

sets_x, sets_y = create_subsets(raw_x, raw_y)

# Dataset 0 where PRI_JET_NUM = 0
x0 = sets_x[0]
y0 = sets_y[0]

# Dataset 1 where PRI_JET_NUM = 1
x1 = sets_x[1]
y1 = sets_y[1]

# Dataset 2 where PRI_JET_NUM = 2
x2 = sets_x[2]
y2 = sets_y[2]

# Dataset 3 where PRI_JET_NUM = 3
x3 = sets_x[3]
y3 = sets_y[3]


### Features Reduction

In [5]:
# if PRI_JET_NUM = 0
# We drop features 4, 5, 6, 11, 12, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29 
x0 = np.delete(x0, [4, 5, 6, 11, 12, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29], 1)

# if PRI_JET_NUM = 1
# We drop features 4, 5, 6, 11, 12, 15, 18, 20, 22, 26, 27, 28 
x1 = np.delete(x1, [4, 5, 6, 11, 12, 15, 18, 20, 22, 26, 27, 28], 1)

# if PRI_JET_NUM == 2 or PRI_JET_NUM == 3
# We drop features 11, 15, 18, 20, 22, 28
x2 = np.delete(x2, [11, 15, 18, 20, 22, 28], 1)
x3 = np.delete(x3, [11, 15, 18, 20, 22, 28], 1)


### Data Standarization Using Power Terms

In [6]:
standardize_x0 = standardize_with_power_terms(x0, 2, True, with_sqrt=True)

standardize_x1 = standardize_with_power_terms(x1, 2, True, with_sqrt=True)

standardize_x2 = standardize_with_power_terms(x2, 2, True, with_sqrt=True)

standardize_x3 = standardize_with_power_terms(x3, 2, True, with_sqrt=True)


# Main Implementations

In [23]:
# Final Datasets
standardize_x = [standardize_x0, standardize_x1, standardize_x2, standardize_x3]
sets_y        = [y0, y1, y2, y3]

# Define the parameters of the algorithm.
max_iters = 5000
gamma     = 0.000002
lambda_   = 0.000001

## 1. Least Squares Using Gradient Descent

In [8]:
ws_1 = []
for x, y in zip(standardize_x, sets_y):
    initial_w = np.zeros(x.shape[1])
    w, loss = least_squares_GD(y, x, initial_w, max_iters, gamma)
    print(np.mean(predict_labels(w, x) == y))
    ws_1.append(w)
    

0.73913304575
0.680697410502
0.704063200937
0.682097094387


## 2. Least Squares Using Stochastic Gradient Descent

In [9]:
ws_2 = []
for x, y in zip(standardize_x, sets_y):
    initial_w = np.zeros(x.shape[1])
    w, loss = least_squares_SGD(y, x, initial_w, max_iters, gamma)
    print(np.mean(predict_labels(w, x) == y))
    ws_2.append(w)
    

0.738942880306
0.680142886619
0.704380793585
0.682818985743


## 3. Least Squares Using Normal Equations

In [10]:
ws_3 = []
for x, y in zip(standardize_x, sets_y):
    w, loss = least_squares(y, x)
    print(np.mean(predict_labels(w, x) == y))
    ws_3.append(w)
    

0.833455105942
0.775572578149
0.804898866591
0.802201768634


## 4. Ridge Regression

In [24]:
ws_4 = []
for x, y in zip(standardize_x, sets_y):
    w, loss = ridge_regression(y, x, lambda_)
    print(np.mean(predict_labels(w, x) == y))
    ws_4.append(w)
    

0.833224905668
0.775753120809
0.804938565672
0.801524995488


## 5. Logistic Regression using Gradient Descent

In [10]:
ws_5 = []
for x, y in zip(standardize_x, sets_y):
    initial_w = np.zeros(x.shape[1])
    w, loss = logistic_regression(y, x, initial_w, max_iters, gamma)
    print(np.mean(predict_labels(w, x) == y))
    ws_5.append(w)
    

0.833455105942
0.775572578149
0.804898866591
0.802201768634


## 6. Regularized Logistic Regression using Gradient Descent

In [10]:
ws_6 = []
for x, y in zip(standardize_x, sets_y):
    initial_w = np.zeros(x.shape[1])
    w, loss = reg_logistic_regression(y, tx, initial_w, max_iters, gamma, lambda_)
    print(np.mean(predict_labels(w, x) == y))
    ws_6.append(w)
    

0.833455105942
0.775572578149
0.804898866591
0.802201768634


In [ ]:
y, x, ind = load_csv_data('higgs-data/test.csv')

y_pred = np.ones(len(y))

mask = x[:, 22] == 0
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w0, stand_x)

mask = x[:, 22] == 1
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w1, stand_x)

mask = x[:, 22] == 2
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w2, stand_x)

mask = x[:, 22] == 3
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w3, stand_x)

create_csv_submission(ind, y_pred, 'prediction.csv')

## ------ below is just used to test -----

In [ ]:
y1 = y*1
x_tr, x_te, y_tr, y_te = split_data(stand_x, y1, 0.8, myseed=1)

for lam in range(100):
    weights = ridge_regression(y_tr, x_tr, lam * 0.01)
    y_pred =  predict_labels(weights, x_te)
    print(np.mean(y_pred == y_te))